# 动态属性和特性

In [5]:
from urllib.request import urlopen
import warnings
import os
import json


URL = 'http://www.oreilly.com/pub/sc/osconfeed'
JSON = 'osconfeed.json'


def load():
    if not os.path.exists(JSON):
        msg = 'downloading {} to {}'.format(URL, JSON)
        warnings.warn(msg)
        with urlopen(URL) as remote, open(JSON, 'wb') as local:
            local.write(remote.read())
    with open(JSON) as fp:
        return json.load(fp)

In [6]:
feed = load()

In [7]:
sorted(feed['Schedule'].keys())

['conferences', 'events', 'speakers', 'venues']

In [8]:
for key, value in sorted(feed['Schedule'].items()):
    print('{:3} {}'.format(len(value), key))

  1 conferences
494 events
357 speakers
 53 venues


In [9]:
feed['Schedule']['speakers'][-1]['name']

'Carina C. Zona'

In [10]:
feed['Schedule']['speakers'][-1]['serial']

141590

In [11]:
feed['Schedule']['events'][40]['name']

'There *Will* Be Bugs'

In [12]:
feed['Schedule']['events'][40]['speakers']

[3471, 5199]

In [13]:
from collections import abc


class FrozenJSON:
    
    def __init__(self, mapping):
        self.__data = dict(mapping)
    
    def __getattr__(self, name):
        if hasattr(self.__data, name):    # 如果name是实例属性__data的属性，返回那个属性。
            # 调用items，keys等方法（数据的属性以及处理数据的方法统称属性）就是通过这种方式处理的。
            return getattr(self.__data, name)
        else:
            return FrozenJSON.build(self.__data[name])
                
    @classmethod
    def build(cls, obj):
        if isinstance(obj, abc.Mapping):
            return cls(obj)
        elif isinstance(obj, abc.MutableSequence):
            return [cls.build(item) for item in obj]
        else:
            return obj

In [14]:
raw_feed = load()
feed = FrozenJSON(raw_feed)

In [15]:
type(feed)

__main__.FrozenJSON

In [16]:
type(feed.Schedule)

__main__.FrozenJSON

In [17]:
type(feed.Schedule.speakers[124])

__main__.FrozenJSON

In [18]:
len(feed.Schedule.speakers)

357

In [19]:
for key, value in sorted(feed.Schedule.items()):
    print('{:3} {}'.format(len(value), key))

  1 conferences
494 events
357 speakers
 53 venues


In [22]:
feed.Schedule.speakers[-1].name

'Carina C. Zona'

In [60]:
talk = feed.Schedule.events[40]
type(talk)

__main__.FrozenJSON

In [61]:
talk.name

'There *Will* Be Bugs'

In [62]:
talk.speakers

[3471, 5199]

In [63]:
talk.flavor

KeyError: 'flavor'

In [66]:
talk.__dict__

{'_FrozenJSON__data': {'serial': 33950,
  'name': 'There *Will* Be Bugs',
  'event_type': '40-minute conference session',
  'time_start': '2014-07-23 14:30:00',
  'time_stop': '2014-07-23 15:10:00',
  'venue_serial': 1449,
  'description': 'If you&#39;re pushing the envelope of programming (or of your own skills)... and even when you’re not... there *will* be bugs in your code.  Don&#39;t panic!  We cover the attitudes and skills (not taught in most schools) to minimize your bugs, track them, find them, fix them, ensure they never recur, and deploy fixes to your users.\r\n',
  'website_url': 'https://conferences.oreilly.com/oscon/oscon2014/public/schedule/detail/33950',
  'speakers': [3471, 5199],
  'categories': ['Python']}}

In [67]:
hasattr(talk, 'items')

True

In [68]:
grad = FrozenJSON({'name': 'Jim Bo', 'class': 1982})

In [69]:
grad.name

'Jim Bo'

In [70]:
grad.class

SyntaxError: invalid syntax (<ipython-input-70-bb5c99ef29c5>, line 1)

In [98]:
getattr(grad, 'class')

AttributeError: type object 'FrozenJSON' has no attribute 'build'

In [100]:
from collections import abc
import keyword


class FrozenJSON:
    
    def __init__(self, mapping):
        self.__data = {}
        for key, value in mapping.items():
            if keyword.iskeyword(key):
                key += '_'
            self.__data[key] = value
    
    def __getattr__(self, name):
        if hasattr(self.__data, name):    # 如果name是实例属性__data的属性，返回那个属性。
            # 调用items，keys等方法（数据的属性以及处理数据的方法统称属性）就是通过这种方式处理的。
            return getattr(self.__data, name)
        else:
            return FrozenJSON.build(self.__data[name])
                
    @classmethod
    def build(cls, obj):
        if isinstance(obj, abc.Mapping):
            return cls(obj)
        elif isinstance(obj, abc.MutableSequence):
            return [cls.build(item) for item in obj]
        else:
            return obj

In [101]:
grad = FrozenJSON({'name': 'Jim Bo', 'class': 1982})
grad.__dict__

{'_FrozenJSON__data': {'name': 'Jim Bo', 'class_': 1982}}

In [102]:
grad.class_

1982

In [103]:
x = FrozenJSON({'2be': 'or not'})    # 如果JSON对象的键不是有效的python标识符，也会出现类似情况
x.2be    # 根据语言的语法判断，2be不是有效的python标识符

SyntaxError: invalid syntax (<ipython-input-103-2298960b07b1>, line 2)

### 使用\_ _ new_ _方法以更灵活的方式创建对象

In [ ]:
def object_maker(the_class, some_arg):
    new_object = the_class.__new__(some_arg)
    if isinstance(new_object, the_class):
        the_class.__init__(new_object, some_arg)
    return new_object

In [104]:
from collections import abc
import keyword


class FrozenJSON:
    
    def __new__(cls, arg):    # __new__是类方法，因此第一个参数是类本身，其余的参数与__init__方法一样
        if isinstance(arg, abc.Mapping):
            return super().__new__(cls)
        elif isinstance(arg, abc.MutableSequence):
            return [cls[item] for item in arg]
        else:
            return arg
        
    def __init__(self, mapping):
        self.__data = {}
        for key, value in mapping.items():
            if keyword.iskeyword(key):
                key += '_'
            self.__data[key] = value
            
    def __getattr__(self, name):
        if hasattr(self.__data, name):
            return getattr(self.__data, name)
        else:
            return FrozenJSON(self.__data[name])

In [105]:
import warnings

DB_NAME = 'schedule1_db'
CONFERENCE = 'conference.115'


class Record:
    def __init__(self, **kwargs):
        self.__dict__.update(kwargs)
    
    
def load_db(db):
    raw_data = load()
    warnings.warn('loading ' + DB_NAME)
    for collection, rec_list in raw_data['Schedule'].items():
        record_type = collection[:-1]
        for record in rec_list:
            key = '{}.{}'.format(record_type, record['serial'])
            record['serial'] = key
            db[key] = Record(**record)

In [106]:
import shelve

db = shelve.open(DB_NAME)
if CONFERENCE not in db:
    load_db(db)

speaker = db['speaker.3471']
type(speaker)

__main__.Record

In [107]:
speaker.name, speaker.twitter

('Anna Ravenscroft', 'annaraven')

In [108]:
speaker.serial

'speaker.3471'

In [109]:
db.close()

## 使用特性获取链接的记录

In [110]:
import warnings
import inspect

DB_NAME2 = 'schedule2_db'
CONFERENCE = 'conference.115'


class Record:
    def __init__(self, **kwargs):
        self.__dict__.update(kwargs)
        
    def __eq__(self, other):
        if isinstance(other, Record):
            return self.__dict__ == other.__dict__
        else:
            return NotImplemented


class MissingDatabaseError(RuntimeError):
    """需要数据库但没有指定数据库时抛出"""
    
    
class DbRecord(Record):
    
    __db = None
    
    @staticmethod
    def set_db(db):
        DbRecord.__db = db
        
    @staticmethod
    def get_db():
        return DbRecord.__db
    
    @classmethod
    def fetch(cls, ident):
        db = cls.get_db()
        try:
            return db[ident]
        except TypeError:
            if db is None:
                msg = "database not set; call '{}.set_db(my_db)'"
                raise MissingDatabaseError(msg.format(cls.__name__))
            else:
                raise
    
    def __repr__(self):
        if hasattr(self, 'serial'):
            cls_name = self.__class__.__name__
            return '<{} serial={!r}>'.format(cls_name, self.serial)
        else:
            return super().__repr__()

        
class Event(DbRecord):
    
    @property
    def venue(self):
        key = 'venue.{}'.format(self.venue_serial)
        return self.__class__.fetch(key)    # 不要写成self.fetch(key)，虽然本示例没有问题
    
    @property
    def speakers(self):    # 这里要联想到特性会覆盖属性！
        if not hasattr(self, '_speaker_objs'):
            spkr_serials = self.__dict__['speakers']    
            # 这里不能写成self.speakers（因为特性的公开名称也是speakers），防止无限递归！
            fetch = self.__class__.fetch
            self._speaker_objs = [fetch('speaker.{}'.format(key)) for key in spkr_serials]
        return self._speaker_objs
    
    def __repr__(self):
        if hasattr(self, 'name'):
            cls_name = self.__class__.__name__
            return '<{} {!r}>'.format(cls_name, self.name)
        else:
            return super().__repr__()

In [111]:
def load_db(db):
    raw_data = load()
    warnings.warn('loading ' + DB_NAME2)
    for collection, rec_list in raw_data['Schedule'].items():
        record_type = collection[:-1]
        cls_name = record_type.capitalize()
        cls = globals().get(cls_name, DbRecord)
        if inspect.isclass(cls) and issubclass(cls, DbRecord):
            factory = cls
        else:
            factory = DbRecord
        for record in rec_list:
            key = '{}.{}'.format(record_type, record['serial'])
            record['serial'] = key
            db[key] = factory(**record)

In [112]:
import shelve

db = shelve.open(DB_NAME2)
if CONFERENCE not in db:
    load_db(db)

In [113]:
DbRecord.set_db(db)

In [114]:
event = DbRecord.fetch('event.33950')

In [115]:
event

<Event 'There *Will* Be Bugs'>

In [116]:
event.venue_serial

1449

In [117]:
event.venue

<DbRecord serial='venue.1449'>

In [118]:
DbRecord.fetch('venue.1449')

<DbRecord serial='venue.1449'>

In [119]:
event.venue.name

'Portland 251'

In [120]:
db['venue.1449'].name

'Portland 251'

In [121]:
Event._DbRecord__db    # 继承自DbRecord类的类属性_DbRecord__db

In [122]:
db['venue.1449'].__dict__    # 实例的__dict__属性保存的是实例属性（包含名称改写的私有实例属性），但不包含私有类属性

{'serial': 'venue.1449',
 'name': 'Portland 251',
 'category': 'Conference Venues'}

In [123]:
for spkr in event.speakers:
    print('{0.serial}: {0.name}'.format(spkr))

speaker.3471: Anna Ravenscroft
speaker.5199: Alex Martelli


In [124]:
dir(event)

['_DbRecord__db',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_speaker_objs',
 'categories',
 'description',
 'event_type',
 'fetch',
 'get_db',
 'name',
 'serial',
 'set_db',
 'speakers',
 'time_start',
 'time_stop',
 'venue',
 'venue_serial',
 'website_url']

In [125]:
db.close()

## 使用特性验证属性

In [126]:
class LineItem:
    
    def __init__(self, description, weight, price):
        self.description = description
        self.weight = weight
        self.price = price
        
    def subtotal(self):
        return self.weight * self.price

In [127]:
raisins = LineItem('Golden raisins', 10, 6.95)
raisins.subtotal()

69.5

In [128]:
raisins.weight = -20
raisins.subtotal()

-139.0

In [129]:
class LineItem:
    
    def __init__(self, description, weight, price):
        self.description = description
        self.weight = weight    # 这一步会调用特性的设值方法，将传入的weight参数赋值给私有属性__weight
        self.price = price
        
    def subtotal(self):
        return self.weight * self.price
    
    @property
    def weight(self):
        return self.__weight
    
    @weight.setter
    def weight(self, value):
        if value > 0:
            self.__weight = value    # 真正的值存储在私有属性__weight中
        else:
            raise ValueError('value must be > 0')    # 验证传入的参数price，如果小于等于0则抛出异常

In [130]:
walnuts = LineItem('walnuts', 0, 10.00)

ValueError: value must be > 0

In [131]:
walnuts = LineItem('walnuts', 5, 10.00)

In [132]:
walnuts.__dict__    # 保存实例属性的__dict__字典中不包含特性

{'description': 'walnuts', '_LineItem__weight': 5, 'price': 10.0}

In [133]:
walnuts.weight

5

In [134]:
walnuts._LineItem__weight = 7

In [135]:
walnuts.weight    # 改变私有属性__weight__后，调用特性的读值方法返回的值也会改变

7

In [136]:
class LineItem:
    
    def __init__(self, description, weight, price):
        self.description = description
        self.weight = weight    # 这一步会调用特性的设值方法，将传入的weight参数赋值给私有属性__weight
        self.price = price
        
    def subtotal(self):
        return self.weight * self.price
    
    def get_weight(self):
        return self.__weight
    
    def set_weight(self, value):
        if value > 0:
            self.__weight = value
        else:
            raise ValueError('value must be > 0')
            
    weight = property(get_weight, set_weight)     
    # 如果没有把函数传给某个参数，那么得到的特性对象(weight)就不允许执行相应的操作！

In [137]:
walnuts = LineItem('walnuts', 5, 10.00)

In [138]:
walnuts.__dict__

{'description': 'walnuts', '_LineItem__weight': 5, 'price': 10.0}

In [139]:
walnuts.weight

5

In [140]:
walnuts._LineItem__weight = 7

In [141]:
walnuts.weight

7

In [142]:
walnuts.weight = 12

In [143]:
walnuts._LineItem__weight

12

In [144]:
walnuts.weight

12

In [145]:
walnuts.weight = 0

ValueError: value must be > 0

### 其实特性就是通过读值方法和设值方法来管理实例属性的存取

In [146]:
class Class:
    data = 'the class attr'
    
    @property
    def prop(self):
        return 'the prop value'

In [147]:
obj = Class()

In [149]:
vars(obj)    # vars返回obj的__dict__属性，表明没有实例属性

{}

In [150]:
obj.data

'the class attr'

In [151]:
obj.data = 'bar'

In [152]:
vars(obj)

{'data': 'bar'}

In [153]:
obj.data

'bar'

In [154]:
Class.data

'the class attr'

In [155]:
Class.prop    # 得到的是一个特性对象

In [156]:
obj.prop

'the prop value'

In [157]:
obj.prop = 'foo'

AttributeError: can't set attribute

In [160]:
obj.__dict__['prop'] = 'foo'

In [161]:
obj.prop

'the prop value'

In [162]:
vars(obj)

{'data': 'bar', 'prop': 'foo'}

In [163]:
Class.prop = 'baz'    # 覆盖Class.prop特性，销毁特性对象

In [164]:
Class.prop

'baz'

In [165]:
obj.prop

'foo'

In [166]:
obj.data

'bar'

In [167]:
Class.data

'the class attr'

In [168]:
Class.data = property(lambda self: 'the "data" prop value')

In [169]:
obj.data    # 现在obj.__class__中寻找attr，今当类中没有名为attr的特性时，才会在obj实例中寻找

'the "data" prop value'

In [170]:
del Class.data

In [172]:
obj.data

'bar'

In [173]:
class LineItem:
    
    def __init__(self, description, weight, price):
        self.description = description
        self.weight = weight    # 这一步会调用特性的设值方法，将传入的weight参数赋值给私有属性__weight
        self.price = price
        
    def subtotal(self):
        return self.weight * self.price
    
    def get_weight(self):
        return self.__weight
    
    def set_weight(self, value):
        if value > 0:
            self.__weight = value
        else:
            raise ValueError('value must be > 0')
            
    weight = property(get_weight, set_weight, doc='weight in kilograms')     
    # 如果没有把函数传给某个参数，那么得到的特性对象(weight)就不允许执行对应的操作！

In [175]:
help(LineItem.weight)

Help on property:

    weight in kilograms



In [180]:
class Foo:
    
    @property
    def bar(self):
        """The bar attribute"""
        return self.__dict__['bar']
    
    @bar.setter
    def bar(self, value):
        self.__dict__['bar'] = value

In [188]:
help(Foo.bar)

Help on property:

    The bar attribute



In [189]:
help(Foo)

Help on class Foo in module __main__:

class Foo(builtins.object)
 |  Data descriptors defined here:
 |  
 |  __dict__
 |      dictionary for instance variables (if defined)
 |  
 |  __weakref__
 |      list of weak references to the object (if defined)
 |  
 |  bar
 |      The bar attribute



In [190]:
def quantity(storage_name):
    
    def qty_getter(instance):
        return instance.__dict__[storage_name]
    
    def qty_setter(instance, value):    # instance指代要把属性存储其中的LineItem实例
        if value > 0:
            instance.__dict__[storage_name] = value
        else:
            raise ValueError('value must be > 0')
            
    return property(qty_getter, qty_setter)


class LineItem:
    weight = quantity('weight')
    # 这里很难避免重复输入，因为特性根本不知道要绑定哪个类属性名来管理实例属性的存取。
    price = quantity('price')
    
    def __init__(self, description, weight, price):
        self.description = description
        self.weight = weight    # 这一步会调用特性的设值方法，将传入的weight参数赋值给私有属性__weight
        self.price = price
        
    def subtotal(self):
        return self.weight * self.price

In [191]:
walnuts = LineItem('walnuts', 5, 10.00)

In [192]:
vars(walnuts)

{'description': 'walnuts', 'weight': 5, 'price': 10.0}

In [193]:
nutmeg = LineItem('Moluccan nutmeg', 8, 13.95)

In [194]:
nutmeg.weight, nutmeg.price

(8, 13.95)

In [195]:
sorted(vars(nutmeg).items())

[('description', 'Moluccan nutmeg'), ('price', 13.95), ('weight', 8)]

In [200]:
class BlackKnight:
    
    def __init__(self):
        self.members = ['an arm', 'another arm',
                        'a leg', 'another leg']
        self.phrases = ["'Tis but a scratch.",
                        "It's just a flesh wound.",
                        "I'm invincible!",
                        "All right, we'll call it a draw."]
    @property
    def member(self):
        print('next member is:')
        return self.members[0]
    
    @member.deleter
    def member(self):
        text = 'BLACK KNIGHT (loses {})\n-- {}'
        print(text.format(self.members.pop(0), self.phrases.pop(0)))

In [201]:
knight = BlackKnight()
knight.member

next member is:


'an arm'

In [202]:
del knight.member

BLACK KNIGHT (loses an arm)
-- 'Tis but a scratch.


In [203]:
del knight.member

BLACK KNIGHT (loses another arm)
-- It's just a flesh wound.


In [204]:
del knight.member

BLACK KNIGHT (loses a leg)
-- I'm invincible!


In [205]:
del knight.member

BLACK KNIGHT (loses another leg)
-- All right, we'll call it a draw.


In [206]:
class BlackKnight:
    
    def __init__(self):
        self.members = ['an arm', 'another arm',
                        'a leg', 'another leg']
        self.phrases = ["'Tis but a scratch.",
                        "It's just a flesh wound.",
                        "I'm invincible!",
                        "All right, we'll call it a draw."]
        
    def member_getter(self):
        print('next member is:')
        return self.members[0]
    
    def member_deleter(self):
        text = 'BLACK KNIGHT (loses {})\n-- {}'
        print(text.format(self.members.pop(0), self.phrases.pop(0)))
        
    member = property(member_getter, fdel=member_deleter)

In [208]:
knight = BlackKnight()
knight.member

next member is:


'an arm'

In [209]:
del knight.member

BLACK KNIGHT (loses an arm)
-- 'Tis but a scratch.


In [210]:
del knight.member

BLACK KNIGHT (loses another arm)
-- It's just a flesh wound.


In [211]:
del knight.member

BLACK KNIGHT (loses a leg)
-- I'm invincible!


In [212]:
del knight.member

BLACK KNIGHT (loses another leg)
-- All right, we'll call it a draw.
